<a href="https://colab.research.google.com/github/shige-ta/dl_sample/blob/main/bayesian_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


train = pd.read_csv('train.csv')
test = pd.read_csv('test')


In [ ]:
target = ''

groups = ''
X_train_ce = train.drop(target,axis=1)
y_train = train[target]
X_train_ce[groups] = pd.factorize(X_train_ce[groups].values)[0]

In [ ]:
x_test = test.drop(target,axis=1)
y_test = test[target]
x_test[groups] = pd.factorize(x_test[groups].values)[0]

In [ ]:
pip install optuna

In [ ]:
pip install category_encoders

In [ ]:
!pip install category_encoders
!pip install bayesian-optimization
!pip install optuna
!pip install pandas

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import time
from matplotlib import pyplot as plt
# from natsort import natsorted 
import time
import lightgbm as lgb
import optuna.integration.lightgbm as opt_lgb
from sklearn.model_selection import GroupKFold
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from bayes_opt import BayesianOptimization

import warnings
warnings.simplefilter('ignore')

In [ ]:
opt_params = bayes_parameter_opt_lgb(
                                    X_train_ce, 
                                    y_train, 
                                    init_round=5, 
                                    opt_round=10, 
                                    n_folds=3, 
                                    random_seed=SEED, 
                                    n_estimators=100, 
                                    learning_rate=0.05
                                    )

In [ ]:
def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
    params = {
        'application':'regression',
        'num_iterations':4000, 
        'learning_rate':0.05, 
        'early_stopping_round':100, 
        'metric':'rmse'
        }

    params["num_leaves"] = round(num_leaves)
    params['feature_fraction'] = max(min(feature_fraction, 1), 0)
    params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
    params['max_depth'] = round(max_depth)
    params['lambda_l1'] = max(lambda_l1, 0)
    params['lambda_l2'] = max(lambda_l2, 0)
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight

    cv_result = lgb.cv(
        params, 
        train_data,
        nfold=n_folds, 
        seed=random_seed, 
        stratified=False, 
        verbose_eval =200, 
        metrics=['rmse']
        )
    
    return max(cv_result['rmse-mean'])

In [ ]:
SEED = 42

lgbBO = BayesianOptimization(
                        lgb_eval, 
                        {
                            'num_leaves': (24, 45),
                            'feature_fraction': (0.1, 0.9),
                            'bagging_fraction': (0.8, 1),
                            'max_depth': (5, 8.99),
                            'lambda_l1': (0, 5),
                            'lambda_l2': (0, 3),
                            'min_split_gain': (0.001, 0.1),
                            'min_child_weight': (5, 50)
                            }, 
                        random_state=SEED
)

In [ ]:
def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=5, random_seed=6, n_estimators=10000, learning_rate=0.05, output_process=False):
    
    # prepare data
    train_data = lgb.Dataset(
                            data=X,  ### 適宜要修正
                            label=y,   ### 適宜要修正
                            free_raw_data=False
                            )

    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        
        params = {
            'application':'regression',
            'num_iterations': n_estimators, 
            'learning_rate':learning_rate, 
            'early_stopping_round':100, 
            'metric':'rmse'
            }

        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight

        cv_result = lgb.cv(
            params, train_data, 
            nfold=n_folds, 
            seed=random_seed, 
            stratified=False, 
            verbose_eval =200, 
            metrics=['rmse']
            )
        
        return max(cv_result['rmse-mean'])

    lgbBO = BayesianOptimization(lgb_eval, 
                                            {'num_leaves': (24, 45),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 8.99),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 3),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    
    # return best parameters
    return lgbBO.res[0]["params"]

In [ ]:


# y_train = y_train.reset_index(drop=True)

gkf = GroupKFold(n_splits=5)

groups = X_train_ce[group]

lgb_results = {}                                    # 学習の履歴を入れる入物

opt_params["application"] = "regression" # 回帰を追加
opt_params["metrics"] = "rmse" # 評価指標を追加
opt_params["num_leaves"] = int(round(opt_params["num_leaves"])) # 整数に変換
opt_params["max_depth"] = int(round(opt_params["max_depth"])) # 整数に変換

cv_result_bayes = []

for train_index, test_index in gkf.split(X_train_ce, y_train, groups):
    X_train_gkf, X_test_gkf = X_train_ce.iloc[train_index], X_train_ce.iloc[test_index]
    y_train_gkf, y_test_gkf = y_train.iloc[train_index], y_train.iloc[test_index]

    # データセットを登録
    lgb_train = lgb.Dataset(X_train_gkf, y_train_gkf)
    lgb_test = lgb.Dataset(X_test_gkf, y_test_gkf, reference=lgb_train)

    lgb_results = {}                                    # 学習の履歴を入れる入物

    model = lgb.train(
                    params=opt_params,           # ベストパラメータをセット
                    train_set=lgb_train,              # 訓練データを訓練用にセット
                    valid_sets=[lgb_train, lgb_test], # 訓練データとテストデータをセット
                    valid_names=['Train', 'Test'],    # データセットの名前をそれぞれ設定
                    num_boost_round=100,              # 計算回数
                    early_stopping_rounds=50,         # アーリーストッピング設定
                    evals_result=lgb_results,
                    verbose_eval=-1,                           # ログを最後の1つだけ表示
                    )
    
    # 損失推移を表示
    loss_train = lgb_results['Train']['rmse']
    loss_test = lgb_results['Test']['rmse']   
    
    fig = plt.figure()
    
    plt.xlabel('Iteration')
    plt.ylabel('logloss')

#     plt.title(f"fold:{fold}")
    plt.plot(loss_train, label='train loss')
    plt.plot(loss_test, label='test loss')
    
    plt.legend()
    plt.show()

    # 推論
    y_pred = model.predict(x_test)

    # 評価
    # rmse = mean_squared_error(y_test, y_pred, squared=False)
    # cv_result_bayes.append(rmse)

print("RMSE:", cv_result_bayes)
print("RMSE:", np.mean(cv_result_bayes))